### **CC57 - MACHINE LEARNING - CLASIFICADOR NAIVE BAYES**

# Detección de fraude con tarjetas de crédito


El conjunto de datos contiene transacciones validas y fraudelentas en realizadas con tarjeta de credito.  

Son 9965 observaciones en 31 variables.
Se desconoce los nombres de las variables ( van del V1 al V28), por tanto, no se sabe que valores representan.

El atributo Time representa la hora en cantidad de segundos, por tanto, habra que convertirlo a horas.

La variable Amount contiene el valor de la transaccion y la variable Class es la eqiqueta o variable objetivo (0 = transaccion real, 1= transaccion fraudulenta).

Se solicita lo siguiente:

- Cargar los datos
- Realizar un analisis visual y preprocesamiento de los datos
- Crear un modelo utilizando el clasificador Naive Bayes (GaussianNB) para predecir si una transaccion con TC en genuina o fraudulenta.  

- Visualizar los resultados del modelo
- Evaluar el modelo



## <font color=blue>0. Cargar Librerias</font>


In [27]:
# Import the necessary packages
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
#from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix
from sklearn.metrics import r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

## <font color=blue>1. Analisis Exploratorio de los Datos</font>


In [28]:
#Cargar archivo de datos
data = pd.read_csv("./infiel.csv")

In [29]:
#Visualizar primeros registros del conjunto de datos
data.head(10)

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666
5,4.0,27.0,9.0,0.0,2.0,14.0,3.0,4.0,4.666666
6,5.0,37.0,23.0,5.5,2.0,12.0,5.0,4.0,0.852174
7,5.0,37.0,23.0,5.5,2.0,12.0,2.0,3.0,1.826086
8,3.0,22.0,2.5,0.0,2.0,12.0,3.0,3.0,4.799999
9,3.0,27.0,6.0,0.0,1.0,16.0,3.0,5.0,1.333333


### **Análisis de las medidas de tendencia/estadistica**

In [30]:
data.describe()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
count,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000,6366.000000
mean,4.109645,29.082862,9.009425,1.396874,2.426170,14.209865,3.424128,3.850141,0.705374
std,0.961430,6.847882,7.280120,1.433471,0.878369,2.178003,0.942399,1.346435,2.203374
min,1.000000,17.500000,0.500000,0.000000,1.000000,9.000000,1.000000,1.000000,0.000000
25%,4.000000,22.000000,2.500000,0.000000,2.000000,12.000000,3.000000,3.000000,0.000000
50%,4.000000,27.000000,6.000000,1.000000,2.000000,14.000000,3.000000,4.000000,0.000000
75%,5.000000,32.000000,16.500000,2.000000,3.000000,16.000000,4.000000,5.000000,0.484848
max,5.000000,42.000000,23.000000,5.500000,4.000000,20.000000,6.000000,6.000000,57.599991


## <font color=blue>2. Preprocesamiento de los datos</font>

### **Verificar datos faltantes y/o duplicados**

In [31]:
# Verificar si hay valores nulos
data.isnull().sum()

rate_marriage      0
age                0
yrs_married        0
children           0
religious          0
educ               0
occupation         0
occupation_husb    0
affairs            0
dtype: int64

### **Separar variables predictoras y el atributo destino de los datos**

In [32]:
# Separar variables predictoras y variable objetivo
X = data.drop(columns=['affairs'])
y = data['affairs'].apply(lambda x: 1 if x > 0 else 0)

## <font color=blue>3. Entrenando el modelo </font>

### **Dividir el conjunto de datos en una proporción de 80: 20**

In [33]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
# Dividir la data en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
print("X_train Shape : ", X_train.shape)
print("X_test Shape  : ", X_test.shape)
print("y_train Shape : ", y_train.shape)
print("y_test Shape  : ", y_test.shape)

X_train Shape :  (5092, 8)
X_test Shape  :  (1274, 8)
y_train Shape :  (5092,)
y_test Shape  :  (1274,)


### Validación cruzada de los datos de entrenamiento

Debido a que los datos están muy desequilibrados, utilizaremos validación cruzada.
* Dividir el conjunto de datos en conjuntos de datos de entrenamiento y prueba.
* Entrene el modelo usando un conjunto de datos de entrenamiento.
* Pruebe el modelo en el conjunto de datos de prueba.

In [49]:
# Encontrar los mejores parámetros con SVC
kernels = list(['linear'])
c = list([5,10,15,20,25,30])
gammas = list([0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])

clf = SVC()
clf.fit(X_train, y_train)

# gridsearch
param_grid = dict(kernel=kernels, C=c, gamma=gammas)
grid = GridSearchCV(clf, param_grid, cv=10, n_jobs=-1)
grid.fit(X_train, y_train)
grid.best_params_

{'C': 25, 'gamma': 0.05, 'kernel': 'linear'}

In [54]:
#Ejecutamos el clasificador SVC con los mejores hioerparametros
clf = SVC(C=25, gamma=0.05, kernel='linear')
clf.fit(X_train, y_train)

SVC(C=25, gamma=0.05, kernel='linear')

## <font color=blue>4. Evaluación y visualización del modelo </font>
<br>

In [56]:
# evaluacion y visualizacion
y_pred = clf.predict(X_test)

print('Accuracy Score: {:.4f}'.format(accuracy_score(y_test, y_pred)))
print('SVC f1-score  : {:.4f}'.format(f1_score(y_pred, y_test, average='weighted')))
print('SVC precision : {:.4f}'.format(precision_score(y_pred, y_test, average='weighted')))
print('SVC recall    : {:.4f}'.format(recall_score(y_pred, y_test, average='weighted')))
print("\n",classification_report(y_pred, y_test))

Accuracy Score: 0.7198
SVC f1-score  : 0.7578
SVC precision : 0.8399
SVC recall    : 0.7198

               precision    recall  f1-score   support

           0       0.93      0.73      0.82      1090
           1       0.30      0.68      0.41       184

    accuracy                           0.72      1274
   macro avg       0.61      0.71      0.61      1274
weighted avg       0.84      0.72      0.76      1274



## **Precision de los datos de entrenamiento**

In [57]:
# Precision de los datos de entrenamiento
y_pred_train = clf.predict(X_train)
print('SVC Train f1-score  : {:.4f}'.format(f1_score(y_pred_train, y_train, average='weighted')))
print('SVC Train precision : {:.4f}'.format(precision_score(y_pred_train, y_train, average='weighted')))
print('SVC Train recall    : {:.4f}'.format(recall_score(y_pred_train, y_train, average='weighted')))
print("\n",classification_report(y_pred_train, y_train))


SVC Train f1-score  : 0.7643
SVC Train precision : 0.8537
SVC Train recall    : 0.7201

               precision    recall  f1-score   support

           0       0.94      0.73      0.82      4461
           1       0.26      0.66      0.37       631

    accuracy                           0.72      5092
   macro avg       0.60      0.69      0.59      5092
weighted avg       0.85      0.72      0.76      5092



## **Precision de los datos de prueba**

In [58]:
# Precision de los datos de test
y_pred_test = clf.predict(X_test)
print('SVC Test f1-score  : {:.4f}'.format(f1_score(y_pred_test, y_test, average='weighted')))
print('SVC Test precision : {:.4f}'.format(precision_score(y_pred_test, y_test, average='weighted')))
print('SVC Test recall    : {:.4f}'.format(recall_score(y_pred_test, y_test, average='weighted')))
print("\n",classification_report(y_pred_test, y_test))

SVC Test f1-score  : 0.7578
SVC Test precision : 0.8399
SVC Test recall    : 0.7198

               precision    recall  f1-score   support

           0       0.93      0.73      0.82      1090
           1       0.30      0.68      0.41       184

    accuracy                           0.72      1274
   macro avg       0.61      0.71      0.61      1274
weighted avg       0.84      0.72      0.76      1274

